In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Concept of conditioning and marginaliizing

Let $p(X, Y)$ be a joint probability distribution of two scalar (or vector) variables $X$ and $Y$. A process of deriving of a probability distribution $p(X)$ is called marginalizing the distribution $p(X, Y)$ for the variable $X$. A process of deriving of a probability distribution $p(X|Y)$ is called conditioning the distribution $p(X, Y)$.

- $p(X, Y)$ -> $p(X)$ - marginalizing
- $p(X, Y)$ -> $p(X|Y)$ - conditioning

In the case of Gaussian $p(X, Y)$, both conditioned and marginalized distributions are also Gaussian, but one needs to determine their mean and covariance matrix.

## Task 1

Let's generate samples from two-variate Gaussian distribution and perform conditioning and marginalizing for it.

Let 
$$ p(x,y) = \mathcal{N}(\begin{bmatrix}x \\ y \\ \end{bmatrix} ; \begin{bmatrix} 0 \\ 0  \end{bmatrix}, \begin{bmatrix}0.3 & 0.1 \\ 0.1 & 0.3 \end{bmatrix} )$$

be a joint distribution of scalar variables x and y. 

Let's generate one million samples from the distibution.

In [ ]:
n = 1_000_000
mean = np.array([0, 0])
cov = np.array([[0.3, 0.1],
                [0.1, 0.3]])
cloud = np.random.multivariate_normal(mean, cov, n)

plt.scatter(cloud[:, 0], cloud[:, 1], s=0.1)
plt.axis("equal")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

Let's marginalize out variable $x$. By that, we estimate statiscics of $y$ variable only.

In [ ]:
y_marginalized = cloud[:, 1]

plt.hist(cloud[:, 1], bins=50)
plt.xlabel("y")
plt.xlim(-3, 3)
plt.show()

We can see that the resulting distribution is also normal. Let's estimate it's parameters from samples.

For gaussians, marginalization results in the position variables to obtained the distribution on the variable $y$:

$$p(y) = \int p(x,y) dx = \mathcal{N}(y; \mu_{y}, \Sigma_{y})$$

So, to marginalize one scalar or vector variables in joint Gaussian distribution, you just need to keep mean and covariance blocks of the variable you want to keep. 

In [ ]:
y_marg_sample_mean = ... # your code here
y_marg_sample_var = ... # your code here

y_marg_analyt_mean = ... # your code here
y_marg_analyt_var = ... # your code here

print(f"Y marginalized sample mean: {y_marg_sample_mean:.5f} \n" 
      f"Y marginalized analytical mean: {y_marg_analyt_mean:.5f}")
print()
print(f"Y marginalized sample variance: {y_marg_sample_var:.5f} \n" 
      f"Y marginalized analytical variance: {y_marg_analyt_var:.5f}")

As we can see, the resulting distribution preserves the same mean and variance for $y$ value as in the original distribution.

Let's condition variable $x$ on $y=0.7$. By that, we estimate statiscics of the $x$ variable, when $y=0.7$.  As $p(y=0.7)=0$, let us create a narrow margin around $y=0.7$.

In [ ]:
y_0 = 1.0
margin = 0.1

# actually should be: condition = (cloud[:, 1] == y_0), but there is no such samples
condition = (np.abs(cloud[:, 1] - y_0) < margin)
x_conditioned = cloud[condition, 0]

fig, axs = plt.subplots(1, 2, figsize=(12, 6))

axs[0].scatter(cloud[:, 0], cloud[:, 1], s=1)
axs[0].axhspan(y_0 - margin, y_0 + margin, color="green", alpha=0.5)
axs[0].axis("equal")
axs[0].set_xlabel("x")
axs[0].set_ylabel("y")

axs[1].hist(x_conditioned, bins=50)
axs[1].set_xlabel(f"x conditioned by y={y_0}")
axs[1].set_xlim(-3, 3)

plt.show()

For Gaussian joint distribution, conditioned distribution has the following analytical form:
$$p(x_a|x_b)=\mathcal{N}(x_a; \mu_a + \Sigma_{ab}\Sigma_b^{-1}(x_b - \mu_b), \Sigma_a - \Sigma_{ab}\Sigma_b^{-1}\Sigma_{ba})$$

In [ ]:
def compute_cond_mean(mu_a, mu_b, x_b, sigma_a, sigma_ab, sigma_b):
    mean = ... # your code here
    return mean

def compute_cond_var(mu_a, mu_b, x_b, sigma_a, sigma_ab, sigma_b):
    cov = ... # your code here
    return cov

In [ ]:
x_conditioned_sample_mean = ... # your code here
x_conditioned_sample_var = ... # your code here

x_conditioned_analyt_mean = ... # your code here
x_conditioned_analyt_var = ... # your code here

print(f"X conditioned sample mean: {x_conditioned_sample_mean:.5f} \n" 
      f"X conditioned analytical mean: {x_conditioned_analyt_mean:.5f}")
print()
print(f"X conditioned sample variance: {x_conditioned_sample_var:.5f} \n" 
      f"X conditioned analytical variance: {x_conditioned_analyt_var:.5f}")

As we can see, both mean and variance have changed. Exact formulas for them will be shown later in this seminar. For further qualitative demonstrations, let's proceed to the [website](https://distill.pub/2019/visual-exploration-gaussian-processes/), section Marginalization and Conditioning.


## Task 2

Suppose that the trajectory of a robot can be described by a vector $p_i=\begin{bmatrix}{x}_i \\ {y}_i \\ \theta_i\end{bmatrix}$ which is a location of the robot $(x,y)$ and a sclar $\theta_i$ which is its heading (in radians).

<img src="heading.png" width="480" height="360">

# 2.1 Marginalization of orientation
Let's assume we have the following Gussian distribution of the 2D pose $p$:

$$ p(\theta, x,y) = \mathcal{N}(\begin{bmatrix}x \\ y \\ \theta\end{bmatrix} ; \begin{bmatrix}1 \\ 1 \\ 0.3 \end{bmatrix}, \begin{bmatrix}0.3 & 0 & 0.1 \\ 0 & 0.2 & 0.2 \\ 0.1 & 0.2 & 0.5  \end{bmatrix} )$$


The problem is to marginalize the position variables to obtained the distribution on the variable $\theta$:

$$p(\theta) = \int \int p(\theta, x,y) dx dy = \mathcal{N}(\theta; \mu_{\theta}, \Sigma_{\theta})$$

In [ ]:
cov = np.array([[0.3, 0, 0.1],
                [0, 0.2, 0.2],
                [0.1, 0.2, 0.5]])
mean = np.array([[1, 1, 0.3]]).T

In [ ]:
# your code in here 
# mean = ...
# cov = ...

**2.2** Marginalize the variable $\theta$
Now, let's calculate the distribution $p(x,y) = \int p(\theta, x,y) d\theta$

In [ ]:
# your code in here 
# mean = ...
# cov = ...

# 3 Conditioning
Now, we have access to the position variables. Calculate the new conditioned distribution $p(\theta | x,y)$ for $x=1.1$ and $y = 1.1$

from class, we just apply the formula as is. $$p(x_a|x_b)=\mathcal{N}(x_a; \mu_a + \Sigma_{ab}\Sigma_b^{-1}(x_b - \mu_b), \Sigma_a - \Sigma_{ab}\Sigma_b^{-1}\Sigma_{ba})$$

In [ ]:
# your code in here 
# mean_theta = ...
# cov_theta =  ...

print('mean_theta', mean_theta)
print('cov_theta ', cov_theta)

**2.2** Condition $p(x,y|\theta)$, knowing that $\theta = 0.4$

In [ ]:
# your code in here 
# mean_xy = ...
# mean_xy =  ...

print('mean_xy', mean_xy)
print('cov_xy ', cov_xy)